In [ ]:
import os
import logging  # noqa: E402
from pathlib import Path  # noqa: E402
import pandas as pd

In [ ]:
# Set atlas unique ID 
unique_atlas_id = ""

# Do not need to set these unless you want the atlas df to print somewhere other than $HOME
atlas_csv_name = f"{unique_atlas_id}_atlas.csv"
atlas_csv_save_location = f"~/{atlas_csv_name}"

In [ ]:
# pylint: disable=wrong-import-position,import-error,missing-class-docstring

class StopExecution(Exception):
    def _render_traceback_(self):
        pass

logger = logging.getLogger("metatlas.jupyter")

kernel_def = """{"argv":["shifter","--entrypoint","--image=ghcr.io/biorack/metatlas/metatlas_shifter:latest","/usr/local/bin/python","-m",
                 "ipykernel_launcher","-f","{connection_file}"],"display_name": "Metatlas Targeted","language": "python",
                 "metadata": { "debugger": true }}"""
kernel_file_name = Path.home() / ".local" / "share" / "jupyter" / "kernels" / "metatlas-targeted" / "kernel.json"
try:
    has_root_kernel = Path("/root/.local/share/jupyter/kernels/papermill/kernel.json").is_file()
except PermissionError:
    has_root_kernel = False
if not has_root_kernel and not kernel_file_name.is_file():
    kernel_file_name.parent.mkdir(parents=True, exist_ok=True)
    with kernel_file_name.open(mode="w", encoding="utf-8") as f:
        f.writelines(kernel_def)
    logger.critical('CRITICAL: Notebook kernel has been installed. Set kernel to "Metatlas Targeted" and re-run notebook.')
    raise StopExecution
try:
    from metatlas.tools import notebook  # noqa: E402
except ImportError as err:
    logger.critical('CRITICAL: Set notebook kernel to "Metatlas Targeted" and re-run notebook.')
    raise StopExecution from err

if not re.match(r'^[a-f0-9]{32}$', unique_atlas_id):
    logger.critical("CRITICAL: Set unique_atlas_id in the first cell to a valid unique identifier hash of the atlas to be used.")

source_code_version_id = None
notebook.setup("INFO", source_code_version_id)

from metatlas.datastructures.utils import get_atlas
from metatlas.io.metatlas_get_data_helper_fun import make_atlas_df

In [ ]:
def atlas_id_to_df(atlas_unique_id: str) -> pd.DataFrame:
    """Retrieve atlas from database using unique id and create DataFrame from compound identification data."""

    logger.info(f"Retrieving atlas with unique ID {atlas_unique_id} from metatlas database...")
    
    atlas = get_atlas(atlas_unique_id)

    atlas_df = make_atlas_df(atlas)

    return atlas_df

def save_atlas_locally(atlas_df: str, save_location: str):
    
    logger.info(f"Saving atlas dataframe to {atlas_csv_save_location}...")
    
    atlas_df.to_csv(atlas_csv_save_location, index=False)

In [ ]:
retrieved_atlas_df = atlas_id_to_df(unique_atlas_id)

In [ ]:
save_atlas_locally(retrieved_atlas_df, atlas_csv_save_location)

In [ ]:
retrieved_atlas_df.head()